<a href="https://colab.research.google.com/github/xingji1337/week6HO/blob/trackc/Week6_3_Streamlit_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 6 — Streamlit App (Graph-RAG + Multi-Hop)
**Goal:** Expose your advanced pipeline via a simple web app.


In [18]:

#@title Write Streamlit app files
import os
APP_DIR = "app_week6_streamlit"
os.makedirs(APP_DIR, exist_ok=True)

app_py = '''import streamlit as st
import time

st.set_page_config(page_title="Week 6: Next-Level RAG", layout="wide")
st.title("Week 6 • Graph-RAG + Multi-Hop Demo")

def graph_rag(query:str):
    time.sleep(0.2)
    return {"answer":"(demo) Graph-RAG answer with [doc1], [doc4].",
            "evidence":[{"doc":"doc1","span":"Method X compared on Dataset D1 (F1=0.78)."},
                        {"doc":"doc4","span":"Paper P3 applies Method X to D2 (Acc=0.82)."}],
            "neighbors":[("Method X","evaluated_on","Dataset D1"),("Method X","reported_in","Paper P3")],
            "trace":[("route","Graph-RAG"),("retrieve","6 spans"),("synthesize","ok")],
            "latency":0.45,"tokens_in":950,"tokens_out":160}

def multi_hop(query:str):
    time.sleep(0.35)
    return {"final":"(demo) Paper P3 ; Dataset D1",
            "subqs":["Which paper introduced Method X?","Which dataset did that paper use for F1?"],
            "hops":[{"subq":"Which paper introduced Method X?","answer":"Paper P3","citations":["doc4"]},
                    {"subq":"Which dataset did that paper use for F1?","answer":"Dataset D1","citations":["doc1"]}],
            "trace":[("decompose","2 hops"),("hop1","ok"),("hop2","ok")]}

with st.sidebar:
    st.header("Settings")
    mode = st.selectbox("Mode", ["Graph-RAG","Multi-Hop"])
    show_graph = st.checkbox("Show neighbor graph (demo)", value=True)

q = st.text_area("Ask a question:", "Which dataset did the paper that introduced Method X use for F1?")
if st.button("Run") and q.strip():
    if mode == "Graph-RAG":
        out = graph_rag(q)
        st.subheader("Answer"); st.write(out["answer"])
        st.markdown("**Evidence**")
        for e in out["evidence"]:
            st.markdown(f"- **{e['doc']}**: {e['span']}")
        if show_graph:
            st.markdown("**Neighbor Triples (demo)**")
            for h,r,t in out["neighbors"]:
                st.write(f"{h} —[{r}]→ {t}")
        st.markdown("**Trace**")
        for s,i in out["trace"]:
            st.write(f"- {s}: {i}")
        st.caption(f"Latency: {out['latency']}s | Tokens: in {out['tokens_in']}, out {out['tokens_out']}")
    else:
        res = multi_hop(q)
        st.subheader("Final"); st.write(res["final"])
        st.markdown("**Sub-Questions & Citations**")
        for i, h in enumerate(res["hops"], 1):
            st.write(f"**Hop {i}** — {h['subq']} ⇒ {h['answer']} (citations: {', '.join(h['citations'])})")
        st.markdown("**Trace**")
        for s,i in res["trace"]:
            st.write(f"- {s}: {i}")
'''
with open(os.path.join(APP_DIR, "app.py"), "w") as f:
    f.write(app_py)

with open(os.path.join(APP_DIR, "requirements.txt"), "w") as f:
    f.write("streamlit\npyyaml\nnetworkx\n")

print("Wrote app files to:", APP_DIR)


Wrote app files to: app_week6_streamlit


In [ ]:

#@title (Optional) Colab launch snippet
print("In Colab, run the following in a cell to launch:")
print("!pip install -q streamlit pyngrok")
print("from pyngrok import ngrok")
print("public_url = ngrok.connect(8501).public_url; print('Public URL:', public_url)")
print("!streamlit run app_week6_streamlit/app.py --server.port 8501 --server.headless true")


In Colab, run the following in a cell to launch:
!pip install -q streamlit pyngrok
from pyngrok import ngrok
public_url = ngrok.connect(8501).public_url; print('Public URL:', public_url)
!streamlit run app_week6_streamlit/app.py --server.port 8501 --server.headless true


In [20]:
!pip install -q streamlit pyngrok
# from pyngrok import ngrok # Commented out ngrok
# public_url = ngrok.connect(8501).public_url; print('Public URL:', public_url) # Commented out ngrok connection

# Use cloudflared instead
!streamlit run app_week6_streamlit/app.py --server.port 8501 --server.headless true &>/content/streamlit.log &
!./cloudflared-linux-amd64 tunnel --url http://localhost:8501 --no-autoupdate

2025-10-07T00:41:38Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-07T00:41:38Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-07T00:41:41Z INF +--------------------------------------------------------------------------------------------+
2025-10-07T00:41:41Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-07T00:41:41Z INF |  https://wrestling-source-quantity-discrete.trycloudfl

In [ ]:
!pip -q install streamlit==1.38.0
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

In [17]:
!pip -q install streamlit==1.38.0
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

# move into the folder that contains app_track_c.py and modules/
# (if your files are on Drive, mount and cd there)
# Example:
# from google.colab import drive; drive.mount('/content/drive')
# %cd /content/drive/MyDrive/week6

# start streamlit (background) and open a public URL
!streamlit run app_track_c.py &>/content/streamlit.log &
!./cloudflared-linux-amd64 tunnel --url http://localhost:8501 --no-autoupdate

2025-10-07T00:03:21Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-07T00:03:21Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-07T00:03:25Z INF +--------------------------------------------------------------------------------------------+
2025-10-07T00:03:25Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-07T00:03:25Z INF |  https://resolution-packed-enclosed-automobiles.tryclo